In [ ]:
!pip install flask line-bot-sdk

In [ ]:
!pip install haversine

In [ ]:
import json

from flask import Flask, request, abort
from haversine import haversine

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )
        

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):    
    m = {'臺北氣象站': [25.037658, 121.514853],
         '士林氣象站': [25.090253, 121.503019],
         '新莊氣象站': [25.051478, 121.446756],
         '三重氣象站': [25.057373, 121.500625],
         '蘆洲氣象站': [25.086594, 121.472331]
        }
    
    r = sorted(m, key=lambda k: haversine((event.message.latitude, event.message.longitude), m[k]))[0]

    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[LocationMessage(title=r, address=r, latitude=m[r][0], longitude=m[r][1])]
        )
    )

if __name__ == "__main__":
    app.run()